# Auction house performance review

---

## The brief

> **Client:** Stockholms Auktionsverk Magasin 5
>
> **Your role:** Junior data analyst at a consulting firm
>
> **Context:** Stockholms Auktionsverk Magasin 5 is one of Sweden's largest auction houses. They've hired your firm to analyze their competitive position and identify growth opportunities.
>
> You have access to historical auction data from Auctionet covering multiple auction houses across Sweden.
>
> **Your task:** Deliver a data-driven competitive analysis. Your analysis should produce clear, well-formatted tables and include written insights interpreting your findings.

---

## Available data

Download the data files from Google Drive and place them in the same folder as this notebook:
- `lab_auctions.csv`
- `lab_bids.csv`

<details>
<summary><strong>Click here for full column reference</strong></summary>

### Auctions dataset (lab_auctions.csv)
| Column | Description |
|--------|-------------|
| `url` | Full URL to the auction item page |
| `id` | Unique internal identifier for the item (Auctionet ID) |
| `auction_id` | Unique identifier for the auction event this item belongs to |
| `currency` | Currency used for bidding (e.g., SEK, EUR) |
| `reserve_met` | Boolean indicating whether the reserve price has been met |
| `estimate` | Estimated price (lower bound) |
| `upper_estimate` | Estimated price (upper bound) |
| `next_bid_amount` | The minimum next bid amount required |
| `state` | Current state of the auction (e.g., active, ended) |
| `hammered` | Boolean, has the item been at a physical auction |
| `processed` | Boolean, has the auction been fully processed by the system? |
| `publicly_visible` | Boolean, is the auction publicly visible? |
| `ends_at` | Timestamp of when the auction ends |
| `ends_at_string` | Human-readable string for end time |
| `ends_at_string_short` | Short formatted version of end time |
| `ends_at_string_short_with_time` | Short formatted version including time |
| `license_weapon` | Boolean, item requires a weapons license |
| `online_auction` | Boolean, indicates if the auction is online |
| `hammer_auction` | Boolean, indicates if the auction is a physical hammer auction |
| `live_bidding_event_starts_at` | Start time for live bidding event |
| `live_bidding_event_approximate_ends_at` | Approximate end time for live event |
| `location` | Physical location of the item |
| `calendar_days_left` | Number of calendar days left before auction ends |
| `title` | Title of the item |
| `description` | Full description of the item |
| `condition` | Condition report |
| `house` | Auction house responsible for the item |
| `number_of_visits` | Number of times the item page has been visited |
| `main_category` | Main category of the item |
| `sub_category` | Sub-category of the item |
| `catalog_nr` | Catalog number for the item |
| `address` | Address where the item is located |
| `placement` | Item placement or lot number within the auction |
| `error` | Error status if any occurred in data retrieval |

### Bids dataset (lab_bids.csv)
| Column | Description |
|--------|-------------|
| `auction_id` | Foreign key linking to the auction item |
| `id` | Unique identifier for the bid |
| `bidder` | Identifier or anonymized ID of the bidder |
| `amount` | Amount of the bid |
| `reserve_met` | Boolean, indicates if this bid met the reserve |
| `first_to_meet_non_minimum_reserve_amount` | Boolean, indicates if this bid was the first to meet a non-minimum reserve |
| `auto` | Boolean, indicates if the bid was placed automatically (auto-bid) |
| `time_string_short` | Short formatted timestamp of bid placement |
| `time_string_long` | Long formatted timestamp of bid placement |
| `max_bid_time_string` | Long formatted timestamp of the bidder's max auto-bid submission |
</details>

## 1. Setup and data loading

Import the libraries you need and load the data files.

In [1]:
import pandas as pd

auc = pd.read_csv("auctions.csv")
bid = pd.read_csv("bids.csv") # upload pandas, and files - auction and bid

In [2]:
pd.set_option("display.max_columns", None) # to see every column use set option

In [3]:
auc.head() # to see first 5 rows from auction df

,url,id,auction_id,currency,reserve_met,estimate,upper_estimate,next_bid_amount,state,hammered,processed,publicly_visible,ends_at,ends_at_string,ends_at_string_short,ends_at_string_short_with_time,license_weapon,online_auction,hammer_auction,live_bidding_event_starts_at,live_bidding_event_approximate_ends_at,location,calendar_days_left,title,description,condition,house,number_of_visits,main_category,sub_category,catalog_nr,address,placement,error
0,https://auctionet.com/sv/3412895-stringhylla-f...,3412895.0,6064215.0,SEK,True,400.0,NaN,1001.0,sold,False,True,True,1.713972e+09,24 apr 2024 kl. 17:15 CEST,ons 24 apr,ons 24 apr kl. 17:15 CEST,False,True,False,NaN,NaN,NaN,0.0,"3412895. STRINGHYLLA, FEM DELAR.","Tre hyllplan, längd 80cm, bredd 21cm.Två gavlar.","Bruks- och ytslitage, repor.",Björnssons Auktionskammare,427.0,Möbler,Skåp & Hyllor,NaN,NaN,NaN,NaN
1,https://auctionet.com/sv/2431371-moissanite-ca...,2431371.0,4344683.0,SEK,True,1000.0,NaN,450.0,sold,False,True,True,1.665938e+09,16 okt 2022 kl. 18:41 CEST,sön 16 okt,sön 16 okt kl. 18:41 CEST,False,True,False,NaN,NaN,NaN,0.0,"2431371. MOISSANITE, ca 3 ct.",Rektangulär trappslipad vit moissanite.GLI-cer...,Se certifikat.,Ekenbergs,284.0,Smycken & Ädelstenar,Ädelstenar,NaN,NaN,NaN,NaN
2,https://auctionet.com/sv/2949567-pistol-halvau...,2949567.0,5519009.0,SEK,True,4000.0,NaN,6100.0,sold,False,True,True,1.698571e+09,29 okt 2023 kl. 10:24 CET,sön 29 okt,sön 29 okt kl. 10:24 CET,True,True,False,NaN,NaN,NaN,0.0,"18.2949567. PISTOL, Halvautomatisk, fabrikat R...","Patronantal 10, piparrangemang EP, piplängd 14...",MYCKET GOTT SKICK.,Walter Borg,1.0,Licensvapen,Pistoler,18.0,NaN,NaN,NaN
3,https://auctionet.com/sv/2835116-bordsur-samt-...,2835116.0,5006279.0,SEK,True,6000.0,NaN,7519.0,sold,False,True,True,1.684614e+09,20 maj 2023 kl. 22:24 CEST,lör 20 maj,lör 20 maj kl. 22:24 CEST,False,True,False,NaN,NaN,NaN,0.0,"2835116. BORDSUR samt CASOLETTER, 1 par, Brons...","Urets längd 48 cm, höjd 40 cmCasoletternas höj...",Bruksslitage. Fungerade vid katalogiseringstil...,Varberg Auktionskammare,388.0,Klockor & Ur,Bordsur,NaN,NaN,NaN,NaN
4,https://auctionet.com/sv/2359720-remington-12-...,2359720.0,4155330.0,SEK,True,2000.0,NaN,1700.0,sold,False,True,True,1.661960e+09,31 aug 2022 kl. 17:38 CEST,ons 31 aug,ons 31 aug kl. 17:38 CEST,True,True,False,NaN,NaN,NaN,0.0,"2359720. Remington 12,7x44R #839.",Karbin Svensk enkelskott. Fabrikat HVA mod kar...,Slitet vapen i mindre gott skick.,Mauritz Widforss,885.0,Licensvapen,Kulgevär,NaN,NaN,NaN,NaN


## 2. Data exploration

Get familiar with the data. Check the shape, columns, data types, and look at some sample rows.

In [4]:
auc.info() # check the data types and null values from auction table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576985 entries, 0 to 576984
Data columns (total 34 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   url                                     576957 non-null  object 
 1   id                                      559683 non-null  float64
 2   auction_id                              559683 non-null  float64
 3   currency                                559683 non-null  object 
 4   reserve_met                             559683 non-null  object 
 5   estimate                                559683 non-null  float64
 6   upper_estimate                          585 non-null     float64
 7   next_bid_amount                         559683 non-null  float64
 8   state                                   559683 non-null  object 
 9   hammered                                559683 non-null  object 
 10  processed                               5596

In [5]:
auc.isnull().sum()# to check how many missing values is auc column

url                                           28
id                                         17302
auction_id                                 17302
currency                                   17302
reserve_met                                17302
estimate                                   17302
upper_estimate                            576400
next_bid_amount                            17302
state                                      17302
hammered                                   17302
processed                                  17302
publicly_visible                           17302
ends_at                                    17302
ends_at_string                             17302
ends_at_string_short                       17302
ends_at_string_short_with_time             17302
license_weapon                             17302
online_auction                             17302
hammer_auction                             17302
live_bidding_event_starts_at              576985
live_bidding_event_a

In [6]:
bid.head() # check 5 columns from bid df

,auction_id,id,bidder,amount,reserve_met,first_to_meet_non_minimum_reserve_amount,auto,time_string_short,time_string_long,max_bid_time_string
0,3412895,26747481,8,951,True,NaN,False,"24 apr, 17:12",24 apr 2024 kl. 17:12:30 CEST,"24 apr 2024, 17:12"
1,3412895,26747475,9,901,True,NaN,False,"24 apr, 17:11",24 apr 2024 kl. 17:11:59 CEST,"24 apr 2024, 17:11"
2,3412895,26747465,8,851,True,NaN,False,"24 apr, 17:11",24 apr 2024 kl. 17:11:17 CEST,"24 apr 2024, 17:11"
3,3412895,26747463,9,801,True,NaN,False,"24 apr, 17:11",24 apr 2024 kl. 17:11:11 CEST,"24 apr 2024, 17:11"
4,3412895,26747456,8,751,True,NaN,False,"24 apr, 17:10",24 apr 2024 kl. 17:10:49 CEST,"24 apr 2024, 17:10"


In [7]:
bid.isnull().sum()# to check how many missing values is bids column

auction_id                                        0
id                                                0
bidder                                            0
amount                                            0
reserve_met                                       0
first_to_meet_non_minimum_reserve_amount    4695183
auto                                              0
time_string_short                                 0
time_string_long                                  0
max_bid_time_string                               0
dtype: int64

In [8]:
# merge with bid Dataset
auc_bid = pd.merge(auc,bid, on = "auction_id", how = "inner")

In [9]:
auc_bid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191279 entries, 0 to 191278
Data columns (total 43 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   url                                       191279 non-null  object 
 1   id_x                                      191279 non-null  float64
 2   auction_id                                191279 non-null  float64
 3   currency                                  191279 non-null  object 
 4   reserve_met_x                             191279 non-null  object 
 5   estimate                                  191279 non-null  float64
 6   upper_estimate                            167 non-null     float64
 7   next_bid_amount                           191279 non-null  float64
 8   state                                     191279 non-null  object 
 9   hammered                                  191279 non-null  object 
 10  processed           

In [10]:
auc_bid.head()

,url,id_x,auction_id,currency,reserve_met_x,estimate,upper_estimate,next_bid_amount,state,hammered,processed,publicly_visible,ends_at,ends_at_string,ends_at_string_short,ends_at_string_short_with_time,license_weapon,online_auction,hammer_auction,live_bidding_event_starts_at,live_bidding_event_approximate_ends_at,location,calendar_days_left,title,description,condition,house,number_of_visits,main_category,sub_category,catalog_nr,address,placement,error,id_y,bidder,amount,reserve_met_y,first_to_meet_non_minimum_reserve_amount,auto,time_string_short,time_string_long,max_bid_time_string
0,https://auctionet.com/sv/1393879-borge-mogense...,1393879.0,2492758.0,SEK,True,4000.0,NaN,7000.0,sold,False,True,True,1.593627e+09,1 jul 2020 kl. 20:11 CEST,ons 1 jul,ons 1 jul kl. 20:11 CEST,False,True,False,NaN,NaN,NaN,0.0,"1393879. BØRGE MOGENSEN. Matsalsgrupp, ""Öresun...","Bord, 5 stolar samt 1 iläggsskiva, bordets dia...","Bruksslitage, fläckig bordsskiva samt smärre f...",Helsingborgs Auktionskammare,3.0,Möbler,Matsalsmöbler,NaN,NaN,NaN,NaN,18531641,1,699,True,699.0,False,"24 okt, 21:19",24 okt 2022 kl. 21:19:18 CEST,"24 okt 2022, 21:19"
1,https://auctionet.com/sv/1393879-borge-mogense...,1393879.0,2492758.0,SEK,True,4000.0,NaN,7000.0,sold,False,True,True,1.593627e+09,1 jul 2020 kl. 20:11 CEST,ons 1 jul,ons 1 jul kl. 20:11 CEST,False,True,False,NaN,NaN,NaN,0.0,"1393879. BØRGE MOGENSEN. Matsalsgrupp, ""Öresun...","Bord, 5 stolar samt 1 iläggsskiva, bordets dia...","Bruksslitage, fläckig bordsskiva samt smärre f...",Helsingborgs Auktionskammare,3.0,Möbler,Matsalsmöbler,NaN,NaN,NaN,NaN,18531639,1,671,False,NaN,False,"24 okt, 21:19",24 okt 2022 kl. 21:19:11 CEST,"24 okt 2022, 21:19"
2,https://auctionet.com/sv/1393879-borge-mogense...,1393879.0,2492758.0,SEK,True,4000.0,NaN,7000.0,sold,False,True,True,1.593627e+09,1 jul 2020 kl. 20:11 CEST,ons 1 jul,ons 1 jul kl. 20:11 CEST,False,True,False,NaN,NaN,NaN,0.0,"1393879. BØRGE MOGENSEN. Matsalsgrupp, ""Öresun...","Bord, 5 stolar samt 1 iläggsskiva, bordets dia...","Bruksslitage, fläckig bordsskiva samt smärre f...",Helsingborgs Auktionskammare,3.0,Möbler,Matsalsmöbler,NaN,NaN,NaN,NaN,18531638,1,364,False,NaN,False,"24 okt, 21:19",24 okt 2022 kl. 21:19:06 CEST,"24 okt 2022, 21:19"
3,https://auctionet.com/sv/1849226-ulrika-hydman...,1849226.0,3264671.0,SEK,True,1000.0,NaN,600.0,sold,False,True,True,1.632946e+09,29 sep 2021 kl. 22:15 CEST,ons 29 sep,ons 29 sep kl. 22:15 CEST,False,True,False,NaN,NaN,NaN,0.0,"1849226. ULRIKA HYDMAN VALLIEN. glas, pokal, K...","Höjd 26,5 cm.",Gott skick.,Hälsinglands Auktionsverk,361.0,Glas,Övrigt,NaN,NaN,NaN,NaN,25887337,1,350,True,NaN,False,"6 mar, 16:26",6 mar 2024 kl. 16:26:00 CET,"6 mar 2024, 16:26"
4,https://auctionet.com/sv/1862268-notknappare-s...,1862268.0,3284950.0,SEK,True,420.0,NaN,489.0,sold,False,True,True,1.632510e+09,24 sep 2021 kl. 21:04 CEST,fre 24 sep,fre 24 sep kl. 21:04 CEST,False,True,False,NaN,NaN,NaN,0.0,"1862268. NÖTKNÄPPARE samt TÄRNINGSKASTARE, oli...","Nötknäpparen, 18,5 cm hög.Tärningskastaren, 22...",Båda i begagnat skick.Nötknäpparen tråden någo...,The Auction Hub,312.0,NaN,NaN,NaN,NaN,NaN,NaN,25559330,1,350,True,NaN,False,"15 feb, 18:03",15 feb 2024 kl. 18:03:08 CET,"15 feb 2024, 18:03"


In [11]:
# this columns has most null value so drop these columns
auc_bid.drop( ["url","upper_estimate", "live_bidding_event_starts_at", "live_bidding_event_approximate_ends_at", "catalog_nr", "address", "placement", "error", "first_to_meet_non_minimum_reserve_amount", "location", "ends_at_string", "ends_at_string_short", "ends_at_string_short_with_time"], axis = 1, inplace= True)

In [12]:
auc_bid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191279 entries, 0 to 191278
Data columns (total 30 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id_x                 191279 non-null  float64
 1   auction_id           191279 non-null  float64
 2   currency             191279 non-null  object 
 3   reserve_met_x        191279 non-null  object 
 4   estimate             191279 non-null  float64
 5   next_bid_amount      191279 non-null  float64
 6   state                191279 non-null  object 
 7   hammered             191279 non-null  object 
 8   processed            191279 non-null  object 
 9   publicly_visible     191279 non-null  object 
 10  ends_at              191279 non-null  float64
 11  license_weapon       191279 non-null  object 
 12  online_auction       191279 non-null  object 
 13  hammer_auction       191279 non-null  object 
 14  calendar_days_left   191279 non-null  float64
 15  title            

In [16]:
# convert ends_at columns float to datetime
auc_bid["end_at_dt"] = pd.to_datetime(auc_bid["ends_at"], unit = "s") 

In [19]:
#drop all irrelivent columns which are not necessary for my analysis
auc_bid.drop(["ends_at", "time_string_short","time_string_long","max_bid_time_string"], axis = 1, inplace= True)

In [20]:
auc_bid.sample()

,id_x,auction_id,currency,reserve_met_x,estimate,next_bid_amount,state,hammered,processed,publicly_visible,license_weapon,online_auction,hammer_auction,calendar_days_left,title,description,condition,house,number_of_visits,main_category,sub_category,id_y,bidder,amount,reserve_met_y,auto,end_at_dt
74443,2057563.0,3619661.0,SEK,True,500.0,400.0,sold,False,True,True,False,True,False,0.0,"2057563. LJUSKRONA, 1900-talets andra hälft.",Renoveringsobjekt.,Inga garantier för om komplett.,Växjö Auktionskammare,441.0,Belysning & Lampor,Ljuskronor,27872989,2,450,False,False,2022-02-26 12:50:00


In [28]:
auc_bid.duplicated().any()# if there any duplicated rows

np.False_

---

## 3. Market position

Help the client understand where they stand in the market.

### What is our overall market share?

Calculate the percentage of all auctions that belong to Stockholms Auktionsverk Magasin 5.

In [47]:
total_row= len(auc_bid)

In [48]:
total_row # check length of rows

191279

In [51]:
maga_5 =auc_bid["house"].isin(["Stockholms Auktionsverk Magasin 5"]).sum()  # filter all Stockholms Auktionsverk Magasin 5


In [55]:
# convert into percentage
percentage = maga_5/ total_row*100

In [56]:
percentage.round(1) # round the percentage

np.float64(5.1)

**My findings:**
A total of 5.1% of all auctions were held by Stockholms Auktionsverk Magasin 5.

### How does our market share vary by category?

In which categories do we have the highest share? The lowest?

In [63]:
cat_stat = auc_bid.groupby("main_category").size()

In [77]:
cat_percen = (cat_stat/cat_stat.sum())*100 #category percentage

In [80]:
# sort from descending
highest_cat= cat_percen.sort_values(ascending = False)

In [82]:
highest_cat.head().round(1) # top 5 highest market share

main_category
Smycken & Ädelstenar    18.5
Klockor & Ur            14.8
Allmoge                 10.7
Belysning & Lampor      10.7
Vapen & Militaria       10.5
dtype: float64

In [83]:
lowest_cat= cat_percen.sort_values(ascending = True) # ordering percentage in descending order

In [85]:
lowest_cat.head().round(1)# top 5 lowest category. 

main_category
Mattor & Textil                  0.0
Leksaker                         0.4
Keramik & Porslin                0.5
Böcker, Kartor & Handskrifter    2.2
Silver & Metall                  2.2
dtype: float64

**Your findings:**
Smycken & Ädelstenar– 18.5%--This is the largest category, suggesting high auction activity and interest.
Mattor & Textil which is NIL no represntation  followed by Leksaker which is  0.4%

### Who are our main competitors?

Identify the largest auction houses and compare them to our client.

In [87]:
auc_house = auc_bid.groupby("house").size().sort_values(ascending= False) # groupby all auction house to see there distribution

In [88]:
auc_hou_per = (auc_house/auc_house.sum())*100

In [92]:
auc_hou_per.head().round(1) # this are the top auction house platform

house
Gomér & Andersson Linköping          6.9
Växjö Auktionskammare                5.1
Stockholms Auktionsverk Magasin 5    5.1
Stadsauktion Sundsvall               5.1
Auktionshuset Kolonn                 5.0
dtype: float64

**Your findings:**
Stockholms Auktionsverk Magasin 5 represents around 5.1% of all auctions and same percentage comes from other 2 auction houses aswel.But top house is Gomér & Andersson Linköping represents 6.9% . 

---

## 4. Performance analysis

Dig deeper into how well the client performs compared to competitors.

### How accurate are our price estimates compared to competitors?

Compare the ratio of final sale price to estimate across auction houses. A ratio > 1 means items sell above estimate.

*You'll need to join the bids data to find the final (highest) bid for each auction.*

In [98]:
auc_bid["ratio"]= auc_bid["amount"]/auc_bid["estimate"] #create new column and put the calculation of ratios

In [118]:
ratio_hou= auc_bid.groupby("house")["ratio"].mean().sort_values(ascending=False) 
ratio_hou.round(1) 

house
Barcelona Auctions                          6.3
Gomér & Andersson Linköping                 4.5
Auktionshaus J. Weiner                      4.5
Rushbrooks Auctioneers                      4.2
The Auction Hub                             3.5
                                           ... 
Auktionshaus von Brühl                      0.3
Colombos                                    0.3
Stockholms Auktionsverk Norrköping          0.2
Kunst- und Auktionshaus Schloß Hagenburg    0.2
Auktionshaus Dannenberg                     0.2
Name: ratio, Length: 70, dtype: float64

In [100]:
ratio_1 = ratio_hou[ratio_hou>1.0] # check which are the houses have ration above 1.0

In [103]:
ratio_1.round(1).count() # count them

np.int64(60)

In [104]:
client_rank = ratio_hou.index.get_loc("Stockholms Auktionsverk Magasin 5")+1

In [105]:
client_rank

56

**My findings:**
Out of 70 auction houses, 60 achieve a sales-to-estimate ratio above 1.0, meaning most houses sell items above their estimated value. Barcelona Auctions has the highest ratio, whereas our client ranks 56th



### Which categories attract the most bidding activity?

Calculate the average number of bids per auction, broken down by category.

In [114]:
bid_per_auc = auc_bid.groupby("auction_id").size().reset_index(name="bid_count")  # bid count per auction

In [115]:
bid_per_auc

,auction_id,bid_count
0,12989.0,3
1,20806.0,2
2,24882.0,7
3,24883.0,1
4,24981.0,3
...,...,...
21488,3874011.0,14
21489,3874444.0,3
21490,3874556.0,18
21491,3875372.0,1


In [120]:
auc_cate= auc_bid[["auction_id", "main_category"]].drop_duplicates() # merge to category info

In [121]:
auc_df= bid_per_auc.merge(auc_cate, on = "auction_id")

In [123]:
avg_bids_by_category = auc_df.groupby("main_category")["bid_count"].mean().reset_index() #avg bids per auction category

In [136]:
auc_sort= avg_bids_by_category.round(1).sort_values(by = "bid_count", ascending = False) # sort dscending order to see most bid count according main category

In [138]:
auc_sort.reset_index(drop= True) # to reset index

,main_category,bid_count
0,Licensvapen,9.4
1,Keramik & Porslin,9.2
2,Glas,9.0
3,Vapen & Militaria,9.0
4,Smycken & Ädelstenar,9.0
5,Klockor & Ur,9.0
6,Belysning & Lampor,8.8
7,"Böcker, Kartor & Handskrifter",8.8
8,"Foto, Kameror & Optik",8.8
9,Möbler,8.8


**My findings:**
Focus on the five main categories with average bids above 9 to attract more customer engagement.

### How well do our listings attract visitors?

Compare average visitor numbers across the top auction houses.

In [150]:
avg_visit = (auc_bid.groupby("house")["number_of_visits"].mean().reset_index())

In [151]:
avg_visit["number_of_visits"]= avg_visit["number_of_visits"].astype(int)

In [156]:
avg_visit= avg_visit.sort_values(by="number_of_visits", ascending =False)

In [160]:
avg_visit = avg_visit.reset_index(drop= True)

In [163]:
avg_visit.head(20)

,house,number_of_visits
0,Auktionshaus Dannenberg,869
1,Auction House Old Eminence,770
2,Frufällans Auktionshus,633
3,Stockholms Auktionsverk Norrköping,591
4,Art and Design Auctions,548
5,HannoVerum,524
6,Connoisseur Bokauktioner,516
7,Stockholms Auktionsverk Düsseldorf/Neuss,495
8,Colombos,486
9,The Auction Hub,475


**My findings:**
Our client ranks 19th with 444 visits, while the top five houses attract nearly double that number.

### What growth opportunities exist for our client?

Find categories where the market is large but our client has low presence.

In [165]:
market= auc_bid.groupby("main_category")["auction_id"].count().reset_index(name="market_size") # identify big category 

In [166]:
client= (auc_bid[auc_bid["house"]== "Stockholms Auktionsverk Magasin 5"].groupby("main_category")["auction_id"].count().reset_index(name="client_size")) # filter my client's auction

In [170]:
compare = market.merge(client, on = "main_category", how = "left").fillna(0)

In [173]:
compare["gap"]= compare["market_size"]- compare["client_size"]

In [174]:
grow_area= compare.sort_values(by="gap", ascending= False)

In [176]:
grow_area.reset_index(drop= True)

,main_category,market_size,client_size,gap
0,Smycken & Ädelstenar,31754,1343.0,30411.0
1,Klockor & Ur,25400,1160.0,24240.0
2,Allmoge,18275,899.0,17376.0
3,Belysning & Lampor,18267,1452.0,16815.0
4,Vapen & Militaria,18048,1907.0,16141.0
5,Glas,15335,598.0,14737.0
6,Licensvapen,9486,0.0,9486.0
7,Möbler,9798,488.0,9310.0
8,"Foto, Kameror & Optik",8636,91.0,8545.0
9,"Mynt, Medaljer & Frimärken",7215,122.0,7093.0


**My findings:**
I have identified several areas where the market is strong and my client has significant potential for growth.

### Which competitor should we study for best practices?

Based on analysis, which competitor performs best overall? What could our client learn from them?

**My findings:**
Gomér & Andersson Linköping , Växjö Auktionskammare, Stadsauktion Sundsvall , Auktionshuset Kolonn    these are the highest sell through rate. strong hammer price and high bidding activities. 

Stockholms Auktionsverk Magasin 5 can learn:-
Increase both trust and sell through rate.
Strengthen item presentation my actively engage with customer and present the product in a attractive way like better photography, detailed description. These activities can increase visits and bidding volume. 

### insight

**My findings:**
Items with higher visit counts align with significantly higher hammer prices. In comparison, my client ranks 19th.

---

## Executive summary

- Visitor engagement is a major driver of value where our client rank 19th. 

- Some products customer has high engamenet. Others auction house focus on these products.

- Client rank 56 to identfy price estimates compared to competitors. 

**My recommendations:**

1. Highlight item exposure, early listing times and target marketting.
2. Need to refine pricing strategy
3. Prioritize high performence categories where demand and margin are strong.
4. Consistency in cateloging
5. Engage customer through social media, campaign and engage more personal in customer relationship team. 